In [1]:
PROJECT = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your PROJECT
BUCKET = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for Cloud MLE
TFVERSION = "1.14"                # TF version for CMLE to use

In [2]:
import os
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION

In [3]:
%%bash
if ! gsutil ls -r gs://${BUCKET} | grep -q gs://${BUCKET}/babyweight/trained_model/; then
    gsutil mb -l ${REGION} gs://${BUCKET}
    # copy canonical model if you didn't do previous notebook
    gsutil -m cp -R gs://cloud-training-demos/babyweight/trained_model gs://${BUCKET}/babyweight/trained_model
fi

# Deploy trained model

In [ ]:
%%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"

# Check to see if the model and version already exist, 
# if so, delete them to deploy anew
if gcloud ai-platform models list | grep "$MODEL_NAME \+ $MODEL_VERSION"; then
    echo "Deleting the version '$MODEL_VERSION' of model '$MODEL_NAME'"
    yes | gcloud ai-platform versions delete ${MODEL_VERSION} --model=$MODEL_NAME
    
    echo "Deleting the model '$MODEL_NAME'"
    yes | gcloud ai-platform models delete ${MODEL_NAME}
else 
    echo "The model '$MODEL_NAME' with version '$MODEL_VERSION' does not exist."
fi

In [ ]:
%%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/babyweight/trained_model/export/exporter/ | tail -1)

echo "Deploying the model '$MODEL_NAME', version '$MODEL_VERSION' from $MODEL_LOCATION"
echo "... this will take a few minutes"

gcloud ai-platform models create ${MODEL_NAME} --regions $REGION
gcloud ai-platform versions create ${MODEL_VERSION} \
  --model ${MODEL_NAME} \
  --origin ${MODEL_LOCATION} \
  --runtime-version $TFVERSION

# Use the deployed model to make online predictions

In [5]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = "babyweight"
MODEL_VERSION = "ml_on_gcp"

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict" \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {"Authorization": "Bearer " + token }
data = {
  "instances": [
    {
      "is_male": "True",
      "mother_age": 26.0,
      "plurality": "Single(1)",
      "gestation_weeks": 39
    },
    {
      "is_male": "False",
      "mother_age": 29.0,
      "plurality": "Single(1)",
      "gestation_weeks": 38
    },
    {
      "is_male": "True",
      "mother_age": 26.0,
      "plurality": "Triplets(3)",
      "gestation_weeks": 39
    },
    {
      "is_male": "Unknown",
      "mother_age": 29.0,
      "plurality": "Multiple(2+)",
      "gestation_weeks": 38
    },
  ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{"predictions": [{"predictions": [7.749312400817871]}, {"predictions": [7.230552673339844]}, {"predictions": [6.335566997528076]}, {"predictions": [6.155895709991455]}]}'


# Use model for batch prediction

In [6]:
%%writefile inputs.json
{"is_male": "True", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}
{"is_male": "False", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}

Overwriting inputs.json


In [7]:
%%bash
INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs

gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ai-platform jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \
    --data-format=TEXT \
    --region ${REGION} \
    --input-paths=$INPUT \
    --output-path=$OUTPUT \
    --model=babyweight \
    --version=ml_on_gcp

jobId: babypred_191207_015509
state: QUEUED


Copying file://inputs.json [Content-Type=application/json]...
/ [1 files][  179.0 B/  179.0 B]                                                
Operation completed over 1 objects/179.0 B.                                      
Removing gs://qwiklabs-gcp-ml-49b827b781ab/babyweight/batchpred/outputs/prediction.errors_stats-00000-of-00001#1574987274127462...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/babyweight/batchpred/outputs/prediction.results-00000-of-00001#1574987269958859...
/ [2/2 objects] 100% Done                                                       
Operation completed over 2 objects.                                              
Job [babypred_191207_015509] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe babypred_191207_015509

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs babypred_191207_015509


In [8]:
!gsutil ls gs://$BUCKET/babyweight/batchpred/outputs

CommandException: One or more URLs matched no objects.


In [9]:
!gsutil cat gs://$BUCKET/babyweight/batchpred/outputs/prediction.results*

CommandException: No URLs matched: gs://qwiklabs-gcp-ml-49b827b781ab/babyweight/batchpred/outputs/prediction.results*
